In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
from IPython.display import clear_output

------
<a id="indice"></a>

# Índice

1. **[Coleta](#coleta)**
    * [Twitter Archive](#coleta:twitter-archive)
    * [Image Prediction](#coleta:image-prediction)
    * [Twitter API](#coleta:twitter-api)
    
* **[Avaliação](#avaliacao)**
    * [Twitter Archive](#avaliacao:twitter-archive)
    * [Image Prediction](#avaliacao:image-prediction)
    * [Twitter API](#avaliacao:twitter-api)


------
<a id="coleta"></a>

# Coleta

<a id="coleta:twitter-archive"></a>

## Coleta: Twitter archive

In [2]:
df_twitter_arc = pd.read_csv('data/twitter-archive-enhanced.csv')
df_twitter_arc.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1805,676942428000112642,NaN,NaN,2015-12-16 01:50:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Who leaves the last cupcake just sitting there...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/676942428...,9,10,None,None,None,None,None
876,761004547850530816,NaN,NaN,2016-08-04 01:03:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo and Ty. Bo eats paper and Ty felt l...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/761004547...,11,10,Bo,None,None,None,None


<a id="coleta:image-prediction"></a>

## Coleta: Image prediction

In [3]:
df_prediction = None

r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

if r.status_code is 200:
    df_prediction = pd.read_csv(pd.compat.StringIO(r.text), sep='\t')    
else:    
    print('ERROR: Image prediction request returned {status_code} status code.'.format(status_code = r.status_code))

In [4]:
df_prediction.sample(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,0.033919,False,partridge,0.000052,False
71,667200525029539841,https://pbs.twimg.com/media/CUJfVMPXIAAgbue.jpg,1,Siberian_husky,0.694904,True,malamute,0.232006,True,Eskimo_dog,0.050635,True


<a id="coleta:twitter-api"></a>

## Coleta: Twitter API

In [5]:
# Twitter APP Config
with open('twitter_config.json', 'r', encoding='utf-8') as file:
    app_config = json.load(file)

In [6]:
# Twitter API settings
api_key = app_config['api_key']
api_secret = app_config['api_secret']
access_token = app_config['access_token']
access_secret = app_config['access_secret']

In [7]:
# Connect to Twitter API
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [8]:
json_path = 'data/tweet_json.txt'
error_log = 'data/tweet_error.log'

# Save tweets from Twitter API
if(not os.path.isfile(json_path)):
    print('Aguarde. Dado o volume de requisições, a coleta de tweets pode demorar alguns minutos.')
    tweet_count = 0
    for tweet_id in df_twitter_arc.tweet_id:
        print('{percent}%'.format(percent=int((++tweet_count/df_twitter_arc.tweet_id.size)*100)))
        try:
            status = api.get_status(tweet_id)
            with open(json_path, 'a', newline='\n') as file:
                file.write(f'{json.dumps(status._json)}\n')

        except Exception as err:
            with open(error_log, 'a', newline='\n') as log:
                log.write(f'{str(tweet_id)}: {str(err.args[0])}\n')
            print(f'{str(tweet_id)}: {str(err.args[0])}')
        
        clear_output(wait=True)
else:
    print('Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.')
        

Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.


In [9]:
# Convert JSON data into dict list
tweets = []

with open(json_path, 'r') as file:
    for line in file:
        try:
            tweet = json.loads(line)
            
            if(tweet.get('entities', False)):
                if(tweet['entities'].get('media', False) and tweet['entities']['media'][0].get('media_url', False)):
                    tweets.append({
                        'id': tweet['id'],
                        'created_at': tweet['created_at'],
                        'in_reply_to_status_id': tweet['in_reply_to_status_id'],
                        'is_quote_status': tweet['is_quote_status'],
                        'retweet_count': tweet['retweet_count'],
                        'favorite_count': tweet['favorite_count'],
                        'media_url': tweet['entities']['media'][0]['media_url'],
                        'retweeted': tweet['retweeted'],
                        'favorited': tweet['favorited']
                    })
            
            
        except Exception as e:
            print(e) 
            
        

In [10]:
# create dataframe of tweets
columns = tweets[0].keys()
df_tweets_api = pd.DataFrame(tweets, columns = columns)

------
<a id="avaliacao"></a>

# Avaliação
Como nas requisições a API do Twitter foram encontrados IDs de Tweets inválidos, com erro, a avaliação seguirá uma ordem diferente a de coleta de dados. Iniciando pelos dados requisitados à API, logo em seguida o Twitter Archive e em seguida os dados da análise de previsão da raça do cão.


<a id="avaliacao:twitter-api"></a>

## Avaliação Twitter API Requests

In [11]:
df_tweets_api.describe()

,id,in_reply_to_status_id,retweet_count,favorite_count
count,1.820000e+03,2.200000e+01,1820.000000,1820.000000
mean,7.237270e+17,6.992047e+17,2512.144505,6820.872527
std,5.777841e+16,4.409222e+16,4891.076169,11900.984651
min,6.660209e+17,6.671522e+17,11.000000,0.000000
25%,6.747671e+17,6.747625e+17,536.000000,1363.500000
50%,7.008223e+17,6.799651e+17,1118.000000,3153.000000
75%,7.617466e+17,7.032024e+17,2526.250000,7463.250000
max,8.924206e+17,8.558181e+17,82905.000000,163034.000000


<a id="tweets-api:info"></a>

In [12]:
df_tweets_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820 entries, 0 to 1819
Data columns (total 9 columns):
id                       1820 non-null int64
created_at               1820 non-null object
in_reply_to_status_id    22 non-null float64
is_quote_status          1820 non-null bool
retweet_count            1820 non-null int64
favorite_count           1820 non-null int64
media_url                1820 non-null object
retweeted                1820 non-null bool
favorited                1820 non-null bool
dtypes: bool(3), float64(1), int64(3), object(2)
memory usage: 90.7+ KB


In [13]:
df_tweets_api.sample(5)

,id,created_at,in_reply_to_status_id,is_quote_status,retweet_count,favorite_count,media_url,retweeted,favorited
722,720415127506415616,Thu Apr 14 00:55:25 +0000 2016,NaN,False,1578,4285,http://pbs.twimg.com/media/Cf9tuHUWsAAHSrV.jpg,False,False
70,846042936437604353,Sun Mar 26 16:55:29 +0000 2017,NaN,False,3035,16495,http://pbs.twimg.com/media/C72_iaUVUAEhZSn.jpg,False,False
266,795464331001561088,Mon Nov 07 03:14:10 +0000 2016,NaN,False,25924,52948,http://pbs.twimg.com/ext_tw_video_thumb/795464...,False,False
992,693622659251335168,Sun Jan 31 02:31:43 +0000 2016,NaN,False,389,1607,http://pbs.twimg.com/media/CaA-IR9VIAAqg5l.jpg,False,False
903,701601587219795968,Mon Feb 22 02:57:08 +0000 2016,NaN,False,481,2170,http://pbs.twimg.com/media/CbyW7B0W8AIX8kX.jpg,False,False


* `created_at` como `string`
* `in_reply_to_status_id` como `Float`

<a id="tweets-api:duplicated"></a>

In [30]:
# Verificar se há IDs duplicados
df_tweets_api.id.duplicated().value_counts()

False    1820
Name: id, dtype: int64

In [32]:
# Verificar se há imagens duplicadas
df_tweets_api.media_url.duplicated().value_counts()

False    1759
True       61
Name: media_url, dtype: int64

* Há tweets referenciando a mesma imagem

<a id="avaliacao:twitter-archive"></a>

## Avaliação: Twitter data archive

In [15]:
 df_twitter_arc.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
184,856526610513747968,8.558181e+17,4.196984e+09,2017-04-24 15:13:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...","THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/856526610...,14,10,None,None,None,None,None
1285,708469915515297792,NaN,NaN,2016-03-12 01:49:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bobble. He's a Croatian Galifianakis. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/708469915...,11,10,Bobble,None,None,None,None
1984,672902681409806336,NaN,NaN,2015-12-04 22:17:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jimison. He's stuck in a pot. Damn it ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/672902681...,9,10,Jimison,None,None,None,None
327,833732339549220864,NaN,NaN,2017-02-20 17:37:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @rolltidered: This is Gabby. Now requests t...,8.324344e+17,44667502.0,2017-02-17 03:39:51 +0000,https://twitter.com/rolltidered/status/8324343...,12,10,Gabby,None,None,None,None
107,871762521631449091,NaN,NaN,2017-06-05 16:15:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rover. As part of pupper protocol he h...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/871762521...,12,10,Rover,None,None,pupper,None


In [16]:
df_twitter_arc.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [17]:
df_twitter_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

<a id="twitter-arc:info"></a>
* Dados incompletos, contém apenas 2356 registros dos 5000 anunciados
* `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id` e `retweeted_status_user_id` em `float`
* `timestamp` e `retweeted_status_timestamp` em `string`

In [33]:
# Verificar se há IDs duplicados
df_twitter_arc.tweet_id.duplicated().value_counts()

False    2356
Name: tweet_id, dtype: int64

<a id="twitter-arc:source"></a>

In [20]:
df_twitter_arc.source.sample(5)

403     <a href="http://twitter.com/download/iphone" r...
480     <a href="http://twitter.com/download/iphone" r...
404     <a href="http://twitter.com/download/iphone" r...
1279    <a href="http://twitter.com/download/iphone" r...
441     <a href="http://twitter.com/download/iphone" r...
Name: source, dtype: object

Variável `source` sem valor para a unidade de observação

<a id="twitter-arc:name"></a>

In [49]:
# Verificar nomes
df_twitter_arc.name.sample(10)

1643     Sadie
1299      None
1501      None
2059    Sparky
1530     Geoff
310       None
1236      Kane
465       None
518     Pavlov
890       None
Name: name, dtype: object

In [50]:
df_twitter_arc[df_twitter_arc.name == 'None'].name.count()

745

* Nomes (`name`) preenchidos com valor literal igual a `'None'`, vazio

<a id="twitter-arc:category"></id>

In [44]:
df_twitter_arc.loc[:,'doggo':].sample(10)

,doggo,floofer,pupper,puppo
1279,None,None,pupper,None
700,None,None,None,None
1384,None,None,None,None
725,None,None,None,None
1392,None,None,None,None
447,doggo,None,None,None
2353,None,None,None,None
277,None,None,None,None
420,None,None,None,None
1549,None,None,None,None


Uma variável em várias colunas

In [51]:
# Verificar padrão no preenchimento das classificações
df_twitter_arc.loc[:,'doggo':].nunique()

doggo      2
floofer    2
pupper     2
puppo      2
dtype: int64

<a id="twitter-arc:expanded_urls"></a>

Registros de `expanded_urls` com URLs inválidas, não existentes

In [ ]:
# URLs sem conteúdo
df_twitter_arc.expanded_urls.sample(10)

In [ ]:
df_twitter_arc.expanded_urls.isnull().value_counts()

------
<a id="avaliacao:image-prediction"></a>

## Avaliação: Image prediction

In [55]:
df_prediction.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [56]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [52]:
df_prediction.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1710,818259473185828864,https://pbs.twimg.com/media/C1sKo_QUkAALtkw.jpg,1,miniature_schnauzer,0.367368,True,toy_poodle,0.112479,True,standard_schnauzer,0.095434,True
728,686386521809772549,https://pbs.twimg.com/media/CYaI5aaW8AE8Uyk.jpg,1,Yorkshire_terrier,0.477704,True,silky_terrier,0.171673,True,Australian_terrier,0.088334,True
82,667491009379606528,https://pbs.twimg.com/media/CUNniSlUYAEj1Jl.jpg,1,borzoi,0.852088,True,ice_bear,0.132264,False,weasel,0.005730,False
1852,840632337062862849,https://pbs.twimg.com/media/C6qGphPV4AEKrdc.jpg,1,golden_retriever,0.711148,True,cocker_spaniel,0.157929,True,Labrador_retriever,0.059582,True
1694,816091915477250048,https://pbs.twimg.com/media/C1NXQ6NXUAEAxIQ.jpg,3,Pomeranian,0.967345,True,Samoyed,0.007397,True,papillon,0.006017,True
1551,793135492858580992,https://pbs.twimg.com/media/CwHIg61WIAApnEV.jpg,1,bakery,0.737041,False,saltshaker,0.052396,False,teddy,0.046593,False
1755,824796380199809024,https://pbs.twimg.com/media/CwiuEJmW8AAZnit.jpg,2,gas_pump,0.676439,False,harvester,0.049995,False,swing,0.044660,False
1081,718246886998687744,https://pbs.twimg.com/media/Cfe5tLWXEAIaoFO.jpg,1,Chihuahua,0.354488,True,carton,0.159672,False,Siberian_husky,0.057498,True
587,679062614270468097,https://pbs.twimg.com/media/CWyD2HGUYAQ1Xa7.jpg,2,cash_machine,0.802333,False,schipperke,0.045519,True,German_shepherd,0.023353,True
1472,779834332596887552,https://pbs.twimg.com/media/CtKHLuCWYAA2TTs.jpg,1,golden_retriever,0.993830,True,cocker_spaniel,0.003143,True,Great_Pyrenees,0.000917,True


<a id="image-prediction:p-values"></id>

In [54]:
df_prediction.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
782,Samoyed,Great_Pyrenees,teddy
157,golden_retriever,Tibetan_mastiff,kuvasz
1255,borzoi,whippet,Saluki
897,Dandie_Dinmont,Old_English_sheepdog,otterhound
1909,Great_Dane,Rhodesian_ridgeback,English_foxhound


Valores não padronizados para as variáveis `p1`, `p2` e `p3`

<a id="image-prediction:duplicated-urls"></a>

In [59]:
# Verificar a duplicidade de IDs
df_prediction.tweet_id.nunique()

2075

In [61]:
# Verificar a duplicidade de Imagens analizadas
df_prediction.jpg_url.nunique()

2009

In [60]:
df_prediction.jpg_url.duplicated().value_counts()

False    2009
True       66
Name: jpg_url, dtype: int64

Imagens duplicadadas

------
<a id="assses-notes"></a>

## Notas de avaliação
### Qualidade

#### `df_tweets_api` - Remaining data from Tweets from Twitter API
1. [Data de criação (`created_at`) como `string`](#tweets-api:info)
* [Valor do id na coluna `in_reply_to_status_id` como `float`](#tweets-api:info)
* [Tweets referenciando a mesma imagem, duplicados](#tweets-api:duplicated)


#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, contém apenas **2356** registros dos **5000** anunciados](#twitter-arc:info)
* [Coluna `source` não acrescenta valor à unidade de obsevação](#twitter-arc:source)
* [Nomes (`name`) preenchidos com valor literal igual a `'None'`, vazio](#twitter-arc:name)
* [Coluna `expanded_urls` com linhas não preenchidas e sem valor para a unidade em observação](#twitter-arc:expanded_urls)


#### `df_prediction` - Image prediction
1. [Previsões [`p1`, `p2`, `p3`] com nomes não padronizados](#image-prediction:p-values)
* [Previsões repetidas sobre a mesma imagem](#image-prediction:duplicated-urls)


### Organização

#### `df_tweets_api` - Remaining data from Tweets from Twitter API
-- *Nada a declarar* --


#### `df_twitter_arc` - Twitter archive
1. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#twitter-arc:info)
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string`](#twitter-arc:info)
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#twitter-arc:category)


#### `df_prediction` - Image prediction
-- *Nada a declarar* --


------
<a id="limpeza"></a>

# Limpeza

In [ ]:
df_twitter_arc['source'][0]

------
<a id="assses-notes"></a>

## Limpeza: Definição 

#### `df_tweets_api` - Remaining data from Tweets from Twitter API
1. [Data de criação (`created_at`) como `string`](#tweets-api:info)
    * Converter datas em string para o formato `datetime`
* [Valor do id na coluna `in_reply_to_status_id` como `float`](#tweets-api:info)
    * Converter valores em `float` para `int64`


#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, de **5000**, apenas **2356** estão disponíveis](#twitter-arc:info)
    * OK - Nada a ser feito
* [Coluna `source` apresenta tags HTML](#twitter-arc:source)
* [Coluna `source` não acrescenta valor à unidade de observação](#twitter-arc:source)
    * As tags HTML poderiam ser removidas com a utilização de expressões regulares. O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida
* [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#twitter-arc:name)
    * Aplicar valor '' para nomes iguais a 'None'
* [Coluna `expanded_urls` com linhas não preenchidas](#twitter-arc:expanded_urls)
* [Endereços inválidos em `expanded_urls`](#twitter-arc:expanded_urls)
    * Alguns endereços retornaram erro 404 e outros não preenchidos. Utilizar apenas tweets com referências válidas na API (`df_tweets_api`)
* [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweet_status_id`, `retweet_status_user_id` estão em `float64`](#twitter-arc:info)
    * Utilizar os valores presentes no *dataframe* de requisições da API e garantir dados válidos
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string` e deveriam ser do tipo `datetime`](#twitter-arc:info)
    * Converter valores para `datetime`
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#twitter-arc:category)
    * Reduzir a variável para uma coluna, `category`


#### `df_prediction` - Image prediction
1. [Previsões [`p1`, `p2`, `p3`] com nomes não padronizados
    * Converter os valores para 'lower case' e substituir '_' por espaços
* [Previsões repetidas sobre a mesma imagem](#image-prediction:duplicated-urls)
    * Descartar previsões repetidas




### Organização


#### `df_tweets_api` - Remaining data from Tweets from Twitter API
-- *Nada a declarar* --


#### `df_twitter_arc` - Twitter archive


#### `df_prediction` - Image prediction
-- *Nada a declarar* --




## Limpeza: Twitter archive

In [ ]:
# Copy df for cleaning
df_arch_clean = df_twitter_arc.copy()

2. [Coluna `source` apresenta tags HTML](#twitter-arc:source)
* [Coluna `source` não acrescenta valor à unidade de observação](#twitter-arc:source)
    * As tags HTML poderiam ser removidas com a utilização de expressões regulares. O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida

In [ ]:
# Remoção da coluna `source`
df_arch_clean.drop('source', axis='columns', inplace=True)

#### Teste

In [ ]:
df_arch_clean.columns

4. [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#twitter-arc:name)
    * Aplicar valor '' para nomes iguais a 'None'

In [ ]:
df_arch_clean[df_arch_clean.name == 'None'].name.count()

In [ ]:
# Aplicar '' aos nomes iguais a 'None'
df_arch_clean.name = df_arch_clean.name.replace('None', '')

In [ ]:
df_arch_clean.name = df_arch_clean.name.apply(lambda name: '' if name.lower() == 'none' else name)

#### Teste

In [ ]:
df_arch_clean[df_arch_clean.name == ''].name.count()

5. [Coluna `expanded_urls` com linhas não preenchidas](#twitter-arc:expanded_urls)
* [Endereços inválidos em `expanded_urls`](#twitter-arc:expanded_urls)
    * Alguns endereços retornaram erro 404 e outros não preenchidos. Utilizar apenas tweets com referências válidas na API (`df_tweets_api`)

_**A união com o dataframe com ids válidos na API será a última etapa**_

7. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64` e deveriam ser em `int64`](#twitter-arc:info)
    * Substituir os valores nulos por `-1`
    * Converter o tipo para `int64`
    * Converter para 
    * Converter valores para `int64`

In [ ]:
df_arch_clean[['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id']].astype(int)

## Limpeza: Image prediction 

# Armazenamento

# Relatórios

* Data wrangling efforts
* Analyses and visualizations